# Milestone Project I: Fantasy Football Analysis
## Tristan Morgan and David Brand





#### List of Requirements:
https://umich.instructure.com/courses/823467/pages/reading-team-project-guidelines?module_item_id=4900067

1. Problem formulation
2. Team formation
3. Selection of data sources
4. Project proposal creation
5. Faculty mentor assigned (by the teaching team)
6. Peer feedback on project proposals
7. Final project report + code
8. Peer feedback on project reports


#### Oral Exam Questions to Review:
https://siads593.org/guides/oral_exam_questions/



# Initial Motivation

1.   The initial motivation is to analyze fantasy football data to determine the most meaningful elements for making optimized fantasy football decisions.
2.   We have a desire to do something relatively novel, but understand that this market is extremely saturated with similar projects.



# Data Sets Chosen

1.   Data 1 - ESPN Fantasy Football API
2.   Data 2 - NFL Savant Play by Play Data
3.   Data 3 - Injury Database ___



# Import Global Modules, Packages, or Objects

In [5]:
import pandas as pd
import os
from datetime import datetime
import json
import random
import time
import requests

# Import Data

#### NFL Savant Play by Play Data


In [ ]:
# Load a Downloaded CSV
def load_savant_data(filepath):
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"{filepath} not found. Download CSVs from nflsavant.com first.")

    df = pd.read_csv(filepath)
    return df


#### ESPN Fantasy Football Database via API

To pull any Fantasy Football data, you must first find a list of public leagues. To do this, we request access to league data by passing a league ID via the Fantasy Football API. To get a League ID we generate random integers that are in a reasonable range for fantasy football league IDs and request data for them. The API returns a code that tells us if we were successful or not. Since that is quite inefficient, after we find 5 accessible leagues, we use their league ID's as a seed for the random generator. This improves our hit percentage, and we can realistically get a list of League ID's for this project.

In [ ]:
# Basic Code that's appropriate for the Notebook
from src.data_fetchers.data_collector import FantasyDataCollector

data_collector = FantasyDataCollector()

league_ids_df = pd.read_csv('data/raw/espn/targets_10team_ppr1_season_2024.csv')
league_ids = league_ids_df['league_id'].tolist()

for league_id in league_ids:
    data_collector.extract_league(league_id, 2024)


array([[ 1409356,     2024],
       [ 1308365,     2024],
       [  915297,     2024],
       [ 1363563,     2024],
       [ 1916109,     2024],
       [ 1240358,     2024],
       [ 1167440,     2024],
       [ 1730745,     2024],
       [  848636,     2024],
       [  875841,     2024],
       [ 1392525,     2024],
       [ 1274253,     2024],
       [ 1021918,     2024],
       [  956578,     2024],
       [ 1273743,     2024],
       [ 1153750,     2024],
       [ 1232323,     2024],
       [ 1059541,     2024],
       [ 1636244,     2024],
       [ 1092983,     2024],
       [ 1255906,     2024],
       [ 1085552,     2024],
       [ 1191424,     2024],
       [ 1417336,     2024],
       [ 1372658,     2024],
       [  867300,     2024],
       [ 1237676,     2024],
       [ 1102665,     2024],
       [ 1018842,     2024],
       [ 1465630,     2024],
       [  848867,     2024],
       [ 1216952,     2024],
       [ 1029814,     2024],
       [ 1054810,     2024],
       [  8752

# Clean and Prepare Data

In [ ]:
# Example NFL Savant Filters

def filter_red_zone_plays(________NFL SAVANT DF HERE_______):
    return df[df['YardLine'] <= 20]


def get_target_counts(________NFL SAVANT DF HERE_______):
    targets = df[df['PassAttempt'] == 1]
    return targets['Receiver'].value_counts()

# Initial Data Exploration

# Hypothesis
This is the hypothesis that we want to investigate after seeing the initial data exploration stage.

# Proposed Experiment
This is the methodology that we want to use to further investigate the hypothesis.

# Review Data Sources
At this point, we need to review the imported data sources by answering a few questions
#### Is the current data quality sufficient?
#### Is the current data quantity sufficient?


# Import and Clean New Data if Needed

# Main Data Manipulation Modules and Explanations

# Main Visualization Modules and Explanations

# Results